In [70]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
df = pd.read_csv('NYCTaxiFares.csv')
df.head()

,pickup_datetime,fare_amount,fare_class,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2010-04-19 08:17:56 UTC,6.5,0,-73.992365,40.730521,-73.975499,40.744746,1
1,2010-04-17 15:43:53 UTC,6.9,0,-73.990078,40.740558,-73.974232,40.744114,1
2,2010-04-17 11:23:26 UTC,10.1,1,-73.994149,40.751118,-73.960064,40.766235,2
3,2010-04-11 21:25:03 UTC,8.9,0,-73.990485,40.756422,-73.971205,40.748192,1
4,2010-04-17 02:19:01 UTC,19.7,1,-73.990976,40.734202,-73.905956,40.743115,1


In [73]:
df['fare_class'].value_counts()

0    80000
1    40000
Name: fare_class, dtype: int64

In [74]:
def haversine_distance(df, lat1, long1, lat2, long2):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers
       
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2]-df[lat1])
    delta_lambda = np.radians(df[long2]-df[long1])
     
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c) # in kilometers

    return d

In [75]:
df['dist_km'] = haversine_distance(df,'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')
df.head()

,pickup_datetime,fare_amount,fare_class,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km
0,2010-04-19 08:17:56 UTC,6.5,0,-73.992365,40.730521,-73.975499,40.744746,1,2.126312
1,2010-04-17 15:43:53 UTC,6.9,0,-73.990078,40.740558,-73.974232,40.744114,1,1.392307
2,2010-04-17 11:23:26 UTC,10.1,1,-73.994149,40.751118,-73.960064,40.766235,2,3.326763
3,2010-04-11 21:25:03 UTC,8.9,0,-73.990485,40.756422,-73.971205,40.748192,1,1.864129
4,2010-04-17 02:19:01 UTC,19.7,1,-73.990976,40.734202,-73.905956,40.743115,1,7.231321


In [76]:
df['EDTdate'] = pd.to_datetime(df['pickup_datetime'].str[:19]) - pd.Timedelta(hours=4)
df['Hour'] = df['EDTdate'].dt.hour
df['AMorPM'] = np.where(df['Hour']<12,'am','pm')
df['Weekday'] = df['EDTdate'].dt.strftime("%a")
df.head()

,pickup_datetime,fare_amount,fare_class,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km,EDTdate,Hour,AMorPM,Weekday
0,2010-04-19 08:17:56 UTC,6.5,0,-73.992365,40.730521,-73.975499,40.744746,1,2.126312,2010-04-19 04:17:56,4,am,Mon
1,2010-04-17 15:43:53 UTC,6.9,0,-73.990078,40.740558,-73.974232,40.744114,1,1.392307,2010-04-17 11:43:53,11,am,Sat
2,2010-04-17 11:23:26 UTC,10.1,1,-73.994149,40.751118,-73.960064,40.766235,2,3.326763,2010-04-17 07:23:26,7,am,Sat
3,2010-04-11 21:25:03 UTC,8.9,0,-73.990485,40.756422,-73.971205,40.748192,1,1.864129,2010-04-11 17:25:03,17,pm,Sun
4,2010-04-17 02:19:01 UTC,19.7,1,-73.990976,40.734202,-73.905956,40.743115,1,7.231321,2010-04-16 22:19:01,22,pm,Fri


In [77]:
df['EDTdate'].min()

Timestamp('2010-04-11 00:00:10')

In [78]:
df['EDTdate'].max()

Timestamp('2010-04-24 23:59:42')

In [79]:
df.columns

Index(['pickup_datetime', 'fare_amount', 'fare_class', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'dist_km', 'EDTdate', 'Hour', 'AMorPM', 'Weekday'],
      dtype='object')

In [80]:
cat_cols = ['Hour', 'AMorPM', 'Weekday']
cont_cols = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count', 'dist_km']
y_col = ['fare_class']  # this column contains the labels

In [81]:
# Convert our three categorical columns to category dtypes.
for cat in cat_cols:
    df[cat] = df[cat].astype('category')

In [82]:
df.dtypes

pickup_datetime              object
fare_amount                 float64
fare_class                    int64
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
dist_km                     float64
EDTdate              datetime64[ns]
Hour                       category
AMorPM                     category
Weekday                    category
dtype: object

In [83]:
df['Hour'].head()

0     4
1    11
2     7
3    17
4    22
Name: Hour, dtype: category
Categories (24, int64): [0, 1, 2, 3, ..., 20, 21, 22, 23]

In [84]:
df['AMorPM'].head()

0    am
1    am
2    am
3    pm
4    pm
Name: AMorPM, dtype: category
Categories (2, object): ['am', 'pm']

In [85]:
df['AMorPM'].cat.categories

Index(['am', 'pm'], dtype='object')

In [86]:
df['AMorPM'].head().cat.codes

0    0
1    0
2    0
3    1
4    1
dtype: int8

In [87]:
df['Weekday'].cat.categories

Index(['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'], dtype='object')

In [88]:
df['Weekday'].head().cat.codes

0    1
1    2
2    2
3    3
4    0
dtype: int8

In [89]:
hr = df['Hour'].cat.codes.values
ampm = df['AMorPM'].cat.codes.values
wkdy = df['Weekday'].cat.codes.values

cats = np.stack([hr, ampm, wkdy], 1)

cats[:5]

array([[ 4,  0,  1],
       [11,  0,  2],
       [ 7,  0,  2],
       [17,  1,  3],
       [22,  1,  0]], dtype=int8)

In [90]:
# Convert categorical variables to a tensor
cats = torch.tensor(cats, dtype=torch.int64)
# this syntax is ok, since the source data is an array, not an existing tensor

cats[:5]

tensor([[ 4,  0,  1],
        [11,  0,  2],
        [ 7,  0,  2],
        [17,  1,  3],
        [22,  1,  0]])

In [91]:
# Convert continuous variables to a tensor
conts = np.stack([df[col].values for col in cont_cols], 1)
conts = torch.tensor(conts, dtype=torch.float)
conts[:5]

tensor([[ 40.7305, -73.9924,  40.7447, -73.9755,   1.0000,   2.1263],
        [ 40.7406, -73.9901,  40.7441, -73.9742,   1.0000,   1.3923],
        [ 40.7511, -73.9941,  40.7662, -73.9601,   2.0000,   3.3268],
        [ 40.7564, -73.9905,  40.7482, -73.9712,   1.0000,   1.8641],
        [ 40.7342, -73.9910,  40.7431, -73.9060,   1.0000,   7.2313]])

In [92]:
conts.type()

'torch.FloatTensor'

In [93]:
# Convert labels to a tensor
y = torch.tensor(df[y_col].values).flatten()

y[:5]

tensor([0, 0, 1, 0, 1])

In [94]:
cats.shape

torch.Size([120000, 3])

In [95]:
conts.shape

torch.Size([120000, 6])

In [96]:
y.shape

torch.Size([120000])

In [97]:
# This will set embedding sizes for Hours, AMvsPM and Weekdays
cat_szs = [len(df[col].cat.categories) for col in cat_cols]
emb_szs = [(size, min(50, (size+1)//2)) for size in cat_szs]
emb_szs

[(24, 12), (2, 1), (7, 4)]

In [98]:
# This is our source data
catz = cats[:4]
catz

tensor([[ 4,  0,  1],
        [11,  0,  2],
        [ 7,  0,  2],
        [17,  1,  3]])

In [99]:
# This is passed in when the model is instantiated
emb_szs

[(24, 12), (2, 1), (7, 4)]

In [100]:
#This is assigned inside the __init__() method
selfembeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])
selfembeds

ModuleList(
  (0): Embedding(24, 12)
  (1): Embedding(2, 1)
  (2): Embedding(7, 4)
)

In [101]:
list(enumerate(selfembeds))

[(0, Embedding(24, 12)), (1, Embedding(2, 1)), (2, Embedding(7, 4))]

In [102]:
# This happens inside the forward() method
embeddingz = []
for i,e in enumerate(selfembeds):
    embeddingz.append(e(catz[:,i]))
embeddingz

[tensor([[ 4.0722e-01,  3.6981e-01,  1.8460e-03, -1.1003e-01, -1.2896e+00,
          -3.5832e-01,  1.7271e-01, -3.6011e-01,  1.6803e+00, -4.3154e-01,
          -1.9219e-01,  1.6781e-01],
         [-2.1111e+00,  6.1436e-01,  2.6107e-01, -1.3886e+00, -8.8402e-01,
           1.0639e+00,  4.7633e-01, -8.5040e-01,  1.0893e+00,  3.1694e-01,
           9.6085e-02,  4.7536e-01],
         [-2.3609e-01,  1.4591e+00, -4.4551e-01,  1.3138e+00, -2.2331e-02,
          -1.5095e+00, -5.9488e-01,  8.0279e-01, -7.8546e-01, -8.5735e-02,
           8.0603e-01,  6.5327e-01],
         [ 4.3548e-01, -1.6129e+00,  1.9381e+00,  8.6291e-01, -2.2317e+00,
           1.5343e-01,  8.8983e-01,  4.4951e-01,  1.3572e+00, -6.1228e-01,
          -9.8617e-01,  5.5194e-01]], grad_fn=<EmbeddingBackward0>),
 tensor([[-0.8968],
         [-0.8968],
         [-0.8968],
         [ 0.3527]], grad_fn=<EmbeddingBackward0>),
 tensor([[ 0.3635,  0.4880, -0.1264,  0.0322],
         [ 2.1781, -1.0624,  0.6635,  0.1584],
         [ 2.1

In [103]:
# We concatenate the embedding sections (12,1,4) into one (17)
z = torch.cat(embeddingz, 1)
z

tensor([[ 4.0722e-01,  3.6981e-01,  1.8460e-03, -1.1003e-01, -1.2896e+00,
         -3.5832e-01,  1.7271e-01, -3.6011e-01,  1.6803e+00, -4.3154e-01,
         -1.9219e-01,  1.6781e-01, -8.9684e-01,  3.6354e-01,  4.8800e-01,
         -1.2644e-01,  3.2160e-02],
        [-2.1111e+00,  6.1436e-01,  2.6107e-01, -1.3886e+00, -8.8402e-01,
          1.0639e+00,  4.7633e-01, -8.5040e-01,  1.0893e+00,  3.1694e-01,
          9.6085e-02,  4.7536e-01, -8.9684e-01,  2.1781e+00, -1.0624e+00,
          6.6348e-01,  1.5840e-01],
        [-2.3609e-01,  1.4591e+00, -4.4551e-01,  1.3138e+00, -2.2331e-02,
         -1.5095e+00, -5.9488e-01,  8.0279e-01, -7.8546e-01, -8.5735e-02,
          8.0603e-01,  6.5327e-01, -8.9684e-01,  2.1781e+00, -1.0624e+00,
          6.6348e-01,  1.5840e-01],
        [ 4.3548e-01, -1.6129e+00,  1.9381e+00,  8.6291e-01, -2.2317e+00,
          1.5343e-01,  8.8983e-01,  4.4951e-01,  1.3572e+00, -6.1228e-01,
         -9.8617e-01,  5.5194e-01,  3.5273e-01, -1.5242e+00,  3.0887e-01,
    

In [104]:
# This was assigned under the __init__() method
selfembdrop = nn.Dropout(.4)

In [105]:
z = selfembdrop(z)
z

tensor([[ 6.7869e-01,  6.1636e-01,  3.0767e-03, -1.8339e-01, -0.0000e+00,
         -0.0000e+00,  2.8785e-01, -6.0018e-01,  2.8005e+00, -7.1923e-01,
         -3.2032e-01,  0.0000e+00, -0.0000e+00,  6.0589e-01,  0.0000e+00,
         -2.1073e-01,  5.3601e-02],
        [-3.5185e+00,  1.0239e+00,  4.3511e-01, -0.0000e+00, -1.4734e+00,
          0.0000e+00,  7.9388e-01, -1.4173e+00,  1.8156e+00,  5.2824e-01,
          1.6014e-01,  0.0000e+00, -1.4947e+00,  3.6301e+00, -1.7707e+00,
          1.1058e+00,  2.6400e-01],
        [-0.0000e+00,  2.4318e+00, -0.0000e+00,  0.0000e+00, -3.7218e-02,
         -2.5158e+00, -9.9146e-01,  0.0000e+00, -1.3091e+00, -0.0000e+00,
          0.0000e+00,  0.0000e+00, -1.4947e+00,  3.6301e+00, -1.7707e+00,
          1.1058e+00,  0.0000e+00],
        [ 0.0000e+00, -0.0000e+00,  3.2302e+00,  0.0000e+00, -0.0000e+00,
          0.0000e+00,  1.4830e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00,
         -1.6436e+00,  0.0000e+00,  5.8788e-01, -2.5404e+00,  0.0000e+00,
    

In [106]:
class TabularModel(nn.Module):

    def __init__(self, emb_szs, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_emb = sum((nf for ni,nf in emb_szs))
        n_in = n_emb + n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
        
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x 

In [107]:
torch.manual_seed(33)
model = TabularModel(emb_szs, conts.shape[1], 2, [200,100], p=0.4) # out_sz = 2

In [108]:
model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(24, 12)
    (1): Embedding(2, 1)
    (2): Embedding(7, 4)
  )
  (emb_drop): Dropout(p=0.4, inplace=False)
  (bn_cont): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=23, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [111]:
batch_size = 60000
test_size = 12000

cat_train = cats[:batch_size-test_size]
cat_test = cats[batch_size-test_size:batch_size]
con_train = conts[:batch_size-test_size]
con_test = conts[batch_size-test_size:batch_size]
y_train = y[:batch_size-test_size]
y_test = y[batch_size-test_size:batch_size]

In [158]:
def max_probability_cross_entropy_loss(y_pred, y_true):
        # Calculate cross-entropy loss
        epsilon = 1e-9
        y_pred=torch.softmax(y_pred, dim=-1)
        y_pred = torch.clip(y_pred, epsilon, 1.0 - epsilon)
        y_pred_max=torch.amax(y_pred,dim=1)
        y_true_actual=torch.amax(y_true,dim=1)
        """y_true_actual=torch.tensor(y_true_actual)
        y_pred_max=torch.tensor(y_pred_max)"""
        y_factor=y_true_actual-y_pred_max
        loss = -torch.mean((torch.sum(y_true * torch.log(y_pred), dim=1)*y_factor))
        return loss

In [156]:
y_pred = model(cat_train, con_train)
epsilon = 1e-9
y_pred=torch.softmax(y_pred, dim=-1)
y_pred = torch.clip(y_pred, epsilon, 1.0 - epsilon)
print(y_pred)
y_pred_max=torch.amax(y_pred,dim=1)
print(y_pred_max)

tensor([[0.7772, 0.2228],
        [0.8562, 0.1438],
        [0.5002, 0.4998],
        ...,
        [0.9210, 0.0790],
        [0.7381, 0.2619],
        [0.0410, 0.9590]], grad_fn=<ClampBackward1>)
tensor([0.7772, 0.8562, 0.5002,  ..., 0.9210, 0.7381, 0.9590],
       grad_fn=<AmaxBackward0>)


In [112]:
#one hot encoding of y_train
y_train=F.one_hot(torch.LongTensor(y_train))

In [109]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [170]:
import time
start_time = time.time()

epochs = 100
losses = []

for i in range(epochs):
    i+=1
    y_pred = model(cat_train, con_train)
    loss = max_probability_cross_entropy_loss(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%25 == 1:
        print(f'epoch: {i:3}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'epoch: {i:3}  loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

epoch:   1  loss: 0.04635187
epoch:  26  loss: 0.04317325
epoch:  51  loss: 0.04067256
epoch:  76  loss: 0.03824435
epoch: 100  loss: 0.03604797

Duration: 19 seconds


In [165]:
#one hot encoding of y_test
y_test=F.one_hot(torch.LongTensor(y_test))
print(y_train)

tensor([[1, 0],
        [1, 0],
        [0, 1],
        ...,
        [1, 0],
        [1, 0],
        [0, 1]])


In [166]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    y_val = model(cat_test, con_test)
    loss = max_probability_cross_entropy_loss(y_val, y_test)
print(f'CE Loss: {loss:.8f}')

CE Loss: 0.04579079


In [167]:
y_test = y[batch_size-test_size:batch_size]

In [171]:
rows = 12000
correct = 0
print(f'{"MODEL OUTPUT":26} ARGMAX  Y_TEST')
for i in range(rows):
    print(f'{str(y_val[i]):26} {y_val[i].argmax():^7}{y_test[i]:^7}')
    if y_val[i].argmax().item() == y_test[i]:
        correct += 1
print(f'\n{correct} out of {rows} = {100*correct/rows:.2f}% correct')

MODEL OUTPUT               ARGMAX  Y_TEST
tensor([ 0.7262, -3.0069])    0      0   
tensor([-2.5374,  2.8745])    1      0   
tensor([ 0.7219, -0.9489])    0      0   
tensor([-0.8932,  1.2972])    1      1   
tensor([ 2.9341, -3.3519])    0      0   
tensor([ 0.7556, -3.1746])    0      0   
tensor([ 0.9385, -3.7236])    0      0   
tensor([-2.7606,  3.8518])    1      1   
tensor([ 2.3690, -1.8041])    0      0   
tensor([-0.1597, -0.5338])    0      1   
tensor([ 1.9577, -2.2232])    0      0   
tensor([ 0.2804, -2.3777])    0      0   
tensor([ 1.2344, -1.6949])    0      0   
tensor([-0.7510,  1.9025])    1      1   
tensor([ 1.7051, -1.1369])    0      0   
tensor([-3.6178,  6.1915])    1      1   
tensor([ 2.4623, -3.2073])    0      1   
tensor([ 2.3579, -3.5284])    0      0   
tensor([ 0.7689, -1.2746])    0      0   
tensor([ 1.4980, -1.7991])    0      0   
tensor([-1.6667,  0.8732])    1      1   
tensor([-1.1909,  1.2513])    1      1   
tensor([-1.2280,  1.6068])    1   

tensor([ 0.9318, -1.5535])    0      0   
tensor([ 1.1884, -1.3494])    0      0   
tensor([-1.9898,  1.9793])    1      1   
tensor([ 1.3084, -1.3288])    0      0   
tensor([ 0.8383, -1.5650])    0      0   
tensor([ 1.3077, -1.9847])    0      0   
tensor([-2.2181,  2.1570])    1      1   
tensor([ 1.6922, -0.7656])    0      0   
tensor([ 2.0625, -2.0275])    0      0   
tensor([-0.5442, -0.6030])    0      1   
tensor([0.3337, 0.3019])      0      0   
tensor([-2.1590,  3.5602])    1      1   
tensor([ 0.7325, -0.9797])    0      0   
tensor([ 2.2595, -2.3951])    0      0   
tensor([-3.2683,  2.9955])    1      1   
tensor([ 0.6418, -1.6577])    0      0   
tensor([ 0.4672, -1.3571])    0      0   
tensor([-1.5830,  2.0921])    1      1   
tensor([0.2919, 0.3853])      1      1   
tensor([ 1.8255, -1.8644])    0      0   
tensor([-1.5487,  2.4763])    1      1   
tensor([-0.1768,  0.4539])    1      1   
tensor([-2.5345,  3.5178])    1      1   
tensor([-2.3663,  2.3528])    1   

tensor([ 0.3854, -1.3748])    0      0   
tensor([ 1.2657, -1.5033])    0      0   
tensor([ 1.8668, -1.0029])    0      0   
tensor([-2.9481,  3.2485])    1      1   
tensor([ 1.8181, -2.1442])    0      0   
tensor([ 2.5152, -1.2941])    0      0   
tensor([ 1.1973, -1.5852])    0      0   
tensor([ 1.6058, -3.4648])    0      0   
tensor([ 1.0189, -2.3809])    0      0   
tensor([ 2.0740, -2.2483])    0      0   
tensor([-0.6895,  1.0117])    1      1   
tensor([ 0.5401, -0.9214])    0      0   
tensor([ 1.5568, -0.7912])    0      1   
tensor([ 2.7925, -2.1159])    0      0   
tensor([ 3.8229, -2.5592])    0      0   
tensor([ 1.9441, -1.5008])    0      0   
tensor([-1.1621,  1.7914])    1      1   
tensor([ 1.5028, -1.0025])    0      0   
tensor([-0.1381,  0.4781])    1      1   
tensor([ 2.4529, -1.4648])    0      0   
tensor([0.5537, 0.1167])      0      0   
tensor([ 1.2793, -1.1325])    0      0   
tensor([ 2.0842, -1.8994])    0      0   
tensor([-1.4889,  0.8716])    1   

tensor([-0.0120,  0.6207])    1      1   
tensor([ 1.4993, -2.7810])    0      0   
tensor([ 0.8466, -1.4375])    0      0   
tensor([-1.8778,  1.9958])    1      1   
tensor([ 0.7534, -2.6959])    0      0   
tensor([ 2.5366, -1.8984])    0      0   
tensor([ 1.4002, -2.1660])    0      0   
tensor([ 1.6169, -0.9855])    0      0   
tensor([-2.3228,  2.7868])    1      1   
tensor([ 1.1796, -2.4448])    0      0   
tensor([ 0.3882, -0.5041])    0      0   
tensor([-2.0470,  3.2833])    1      1   
tensor([ 0.6870, -2.4382])    0      0   
tensor([-2.7916,  3.0837])    1      1   
tensor([-1.0002,  0.5398])    1      0   
tensor([ 2.0811, -1.0113])    0      0   
tensor([-2.3121,  4.0045])    1      1   
tensor([-1.9371,  1.3989])    1      0   
tensor([ 0.7537, -1.6714])    0      0   
tensor([ 1.3055, -1.7186])    0      0   
tensor([-1.8390,  2.4525])    1      1   
tensor([ 4.6619, -1.4863])    0      0   
tensor([ 0.7199, -0.4020])    0      0   
tensor([ 1.8130, -1.8932])    0   

tensor([-1.9825,  2.7772])    1      1   
tensor([ 2.4871, -1.9172])    0      0   
tensor([ 2.4945, -2.7966])    0      0   
tensor([ 1.4383, -1.6978])    0      0   
tensor([-0.1394,  0.9712])    1      0   
tensor([ 1.6391, -2.9890])    0      0   
tensor([ 0.7626, -1.6967])    0      0   
tensor([ 1.1192, -1.9963])    0      0   
tensor([-0.7849,  1.5299])    1      0   
tensor([ 0.6657, -2.5634])    0      1   
tensor([-2.8200,  2.9676])    1      1   
tensor([ 1.2942, -1.3505])    0      0   
tensor([ 1.2551, -1.1792])    0      0   
tensor([-0.9602,  2.3855])    1      1   
tensor([-1.4359,  1.3635])    1      0   
tensor([ 1.5296, -0.6558])    0      0   
tensor([ 1.4154, -1.8946])    0      0   
tensor([ 1.4748, -1.3602])    0      0   
tensor([-0.0404, -0.3994])    0      1   
tensor([-3.3273,  4.2570])    1      1   
tensor([-1.6699,  2.0550])    1      1   
tensor([ 1.5854, -2.5345])    0      0   
tensor([ 2.3651, -1.5716])    0      0   
tensor([ 1.0382, -3.0036])    0   

...

tensor([-3.4254,  4.8015])    1      1   
tensor([-1.1601,  3.0613])    1      1   
tensor([ 0.2672, -1.7048])    0      0   
tensor([0.2815, 0.0155])      0      1   
tensor([ 1.6282, -1.3558])    0      0   
tensor([-1.4667,  2.0175])    1      1   
tensor([ 0.9010, -2.1881])    0      0   
tensor([ 1.5350, -1.9893])    0      0   
tensor([ 1.4326, -1.4657])    0      0   
tensor([-3.7370,  4.9937])    1      1   
tensor([-0.3041,  0.1263])    1      0   
tensor([-2.7753,  3.1505])    1      1   
tensor([ 2.5243, -3.0370])    0      0   
tensor([-0.0253,  0.0978])    1      0   
tensor([ 0.7471, -0.2764])    0      1   
tensor([-2.2243,  2.6894])    1      1   
tensor([ 1.5230, -2.1000])    0      0   
tensor([-2.0908,  2.5555])    1      1   
tensor([ 1.8369, -2.6372])    0      0   
tensor([ 1.0169, -2.9918])    0      0   
tensor([ 0.5358, -1.0367])    0      1   
tensor([-2.2194,  0.8184])    1      1   
tensor([ 1.5729, -2.3581])    0      0   
tensor([ 2.3375, -2.8021])    0   

tensor([ 1.2176, -0.0552])    0      0   
tensor([ 1.4688, -2.6120])    0      0   
tensor([ 0.2260, -0.9264])    0      0   
tensor([ 2.3030, -2.2864])    0      0   
tensor([-0.9255,  1.1946])    1      1   
tensor([-1.0627,  1.6877])    1      1   
tensor([ 1.8704, -1.5330])    0      0   
tensor([-0.1369,  0.1461])    1      1   
tensor([ 0.2366, -0.8712])    0      1   
tensor([-0.5677,  0.0037])    1      0   
tensor([-0.9641,  0.7235])    1      1   
tensor([-4.5393,  6.3122])    1      1   
tensor([-0.4114,  0.5772])    1      1   
tensor([-1.6431,  2.2832])    1      1   
tensor([ 1.3761, -0.9574])    0      0   
tensor([ 2.4184, -0.8150])    0      0   
tensor([-1.0684,  0.9616])    1      1   
tensor([-1.0964,  2.5210])    1      1   
tensor([-0.7554,  0.6770])    1      1   
tensor([-1.3267,  1.2853])    1      1   
tensor([ 1.3666, -1.9365])    0      0   
tensor([ 1.0664, -1.6327])    0      0   
tensor([-2.6821,  2.9052])    1      1   
tensor([ 1.9871, -1.1747])    0   

tensor([ 1.2394, -2.1070])    0      0   
tensor([ 1.9882, -1.5774])    0      0   
tensor([-0.0315,  0.6939])    1      1   
tensor([ 0.3278, -0.6454])    0      1   
tensor([-0.0093, -0.0277])    0      0   
tensor([ 2.7510, -1.1566])    0      0   
tensor([-1.4290,  0.4378])    1      1   
tensor([ 2.2357, -1.8554])    0      0   
tensor([0.5574, 0.0839])      0      0   
tensor([-0.5371, -0.6270])    0      0   
tensor([ 0.8362, -0.5359])    0      0   
tensor([ 1.3886, -3.2546])    0      0   
tensor([ 0.3480, -1.2637])    0      0   
tensor([-2.4503,  5.9070])    1      1   
tensor([ 0.9596, -0.7945])    0      0   
tensor([-0.1277, -0.7469])    0      1   
tensor([ 1.3946, -3.3096])    0      0   
tensor([ 0.9202, -0.5760])    0      0   
tensor([ 1.9668, -2.1671])    0      0   
tensor([ 2.1888, -2.5954])    0      0   
tensor([ 2.2928, -2.8075])    0      0   
tensor([ 0.2733, -0.9627])    0      0   
tensor([ 2.4125, -1.6650])    0      0   
tensor([ 2.3187, -1.0348])    0   

tensor([-0.0450,  0.4815])    1      1   
tensor([ 1.0669, -1.8318])    0      0   
tensor([-1.2489,  2.1097])    1      1   
tensor([ 0.9573, -2.0170])    0      0   
tensor([-1.0291,  0.6392])    1      1   
tensor([-0.1660,  0.3976])    1      0   
tensor([ 1.3362, -3.7002])    0      0   
tensor([ 1.7400, -2.3350])    0      0   
tensor([ 1.3651, -3.3506])    0      0   
tensor([ 1.1533, -0.8514])    0      0   
tensor([ 0.8091, -2.2159])    0      0   
tensor([ 0.4931, -1.9473])    0      0   
tensor([ 0.8194, -1.8516])    0      0   
tensor([-2.2472,  3.3325])    1      1   
tensor([ 0.6276, -0.3710])    0      0   
tensor([-1.8030,  2.6181])    1      1   
tensor([-1.9603,  3.3895])    1      1   
tensor([ 1.5409, -1.3423])    0      0   
tensor([ 0.2903, -0.2696])    0      1   
tensor([-1.9569,  1.9869])    1      1   
tensor([ 1.0860, -1.8750])    0      0   
tensor([ 1.1241, -2.5844])    0      0   
tensor([ 1.6321, -1.6658])    0      0   
tensor([-0.0560,  0.0011])    1   

tensor([ 1.0767, -2.1944])    0      0   
tensor([ 2.5569, -2.9828])    0      0   
tensor([ 1.0178, -1.9599])    0      0   
tensor([-2.4706,  2.9642])    1      1   
tensor([-2.5971,  4.9562])    1      1   
tensor([ 2.6577, -1.3452])    0      0   
tensor([-0.2756, -0.9406])    0      1   
tensor([ 1.9691, -2.9381])    0      0   
tensor([-0.9713, -0.1604])    1      0   
tensor([ 0.3653, -1.0110])    0      0   
tensor([ 2.5587, -1.5043])    0      0   
tensor([-0.9018,  1.6006])    1      1   
tensor([ 1.0320, -2.5358])    0      0   
tensor([ 1.0409, -5.2469])    0      0   
tensor([ 1.8755, -2.5189])    0      0   
tensor([-1.8001,  0.7855])    1      1   
tensor([ 2.0343, -1.8853])    0      0   
tensor([-1.2118,  1.5305])    1      1   
tensor([ 1.2920, -1.1191])    0      0   
tensor([ 2.5662, -3.5206])    0      0   
tensor([ 1.8622, -0.9031])    0      0   
tensor([-0.9335,  0.0735])    1      0   
tensor([ 2.9914, -2.3457])    0      0   
tensor([ 1.3514, -1.3634])    0   